In [1]:
!rm -r HoMM
!git clone https://github.com/tkukurin/HoMM.git

Cloning into 'HoMM'...
remote: Enumerating objects: 763, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 763 (delta 126), reused 231 (delta 102), pack-reused 503
Receiving objects: 100% (763/763), 198.41 KiB | 6.01 MiB/s, done.
Resolving deltas: 100% (369/369), done.


In [2]:
import sys  # haxx so github imported stuff works

def _haxx(x):
  if x not in sys.path:
    sys.path.append(x)

_haxx('HoMM')
from HoMM import HoMM_model

_haxx('HoMM/examples')
import polynomials

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np

import functools as ft
import itertools as it
import copy

import numpy as np
import tensorflow as tf

import re

# C/p of example

In [ ]:
from HoMM import HoMM_model
from HoMM.configs import default_run_config, default_architecture_config

run_config = {
  **default_run_config.default_run_config,
  "output_dir": "/data2/lampinen/polynomials_results/",
  
  "num_base_train_tasks": 60, # prior to meta-augmentation
  "num_base_eval_tasks": 40, # prior to meta-augmentation

  "num_variables": 4,
  "max_degree": 2,
  "poly_coeff_sd": 2.5,
  "point_val_range": 1,

  "meta_add_vals": [-3, -1, 1, 3],
  "meta_mult_vals": [-3, -1, 3],
  "new_meta_tasks": [],
  "new_meta_mappings": ["add_%f" % 2., "add_%f" % -2., "mult_%f" % 2., "mult_%f" % -2.],

  "run_offset": 2,
  "num_runs": 3,
  
  "num_epochs": 5000,
}

architecture_config = {
  **default_architecture_config.default_architecture_config,
  "task_conditioned_not_hyper": False,
}


class poly_HoMM_model(HoMM_model.HoMM_model):
  def __init__(self, run_config=None):
    super(poly_HoMM_model, self).__init__(
      architecture_config=architecture_config, run_config=run_config)

  def _pre_build_calls(self):
    # set up the base tasks
    poly_fam = polynomials.polynomial_family(
      self.run_config["num_variables"], self.run_config["max_degree"])
    self.run_config["variables"] = poly_fam.variables

    self.base_train_tasks = [
     poly_fam.sample_polynomial(coefficient_sd=self.run_config["poly_coeff_sd"])
     for _ in range(self.run_config["num_base_train_tasks"])
    ]
    self.base_eval_tasks = [
     poly_fam.sample_polynomial(coefficient_sd=self.run_config["poly_coeff_sd"])
     for _ in range(self.run_config["num_base_eval_tasks"])
    ]

    # set up the meta tasks

    self.meta_class_train_tasks = [
     "is_constant_polynomial", "is_intercept_nonzero", 
     *[f"is_{var}_relevant" for var in self.run_config["variables"]]
    ]
    self.meta_class_eval_tasks = [] 

    add = [f"add_{c}" for c in self.run_config["meta_add_vals"]]
    mul = [f"mult_{c}" for c in self.run_config["meta_mult_vals"]]
    self.meta_map_train_tasks = ["square"] + add + mul
    self.meta_map_eval_tasks = self.run_config["new_meta_mappings"] 

    permutation_mappings = [
     "permute_" + "".join(map(str, p))
     for p in it.permutations(range(self.run_config["num_variables"]))
    ]

    np.random.seed(0)
    np.random.shuffle(permutation_mappings)
    self.meta_map_train_tasks += permutation_mappings[:len(permutation_mappings)//2]
    self.meta_map_eval_tasks += permutation_mappings[len(permutation_mappings)//2:]

    # set up language

    #vocab = ['PAD'] + [str(x) for x in range(10)] + [".", "+", "-", "^"] + poly_fam.variables
    #vocab_to_int = dict(zip(vocab, range(len(vocab))))
    #
    #self.run_config["vocab"] = vocab

    # set up the meta pairings 
    self.meta_pairings, implied_tasks_train_tasks, implied_tasks_eval_tasks = polynomials.get_meta_pairings(
      base_train_tasks=self.base_train_tasks,
      base_eval_tasks=self.base_eval_tasks,
      meta_class_train_tasks=self.meta_class_train_tasks,
      meta_class_eval_tasks=self.meta_class_eval_tasks,
      meta_map_train_tasks=self.meta_map_train_tasks,
      meta_map_eval_tasks=self.meta_map_eval_tasks) 

    # add the base tasks implied by the mappings
    self.base_train_tasks += implied_tasks_train_tasks

    # in general, we want evaluation to be of ability to metamap a trained
    # task to a held-out one, so we move the meta-mapping eval sources to
    # train, and use the implied tasks as eval
    self.base_train_tasks += self.base_eval_tasks 
    self.base_eval_tasks = implied_tasks_eval_tasks

  def fill_buffers(self, num_data_points=1):
    """Add new "experiences" to memory buffers."""
    this_tasks = self.base_train_tasks + self.base_eval_tasks 
    for t in this_tasks:
      buff = self.memory_buffers[polynomials.stringify_polynomial(t)]
      x_data = np.zeros([num_data_points] + self.architecture_config["input_shape"])
      y_data = np.zeros([num_data_points] + self.architecture_config["output_shape"])
      for point_i in range(num_data_points):
        point = t.family.sample_point(val_range=self.run_config["point_val_range"])
        x_data[point_i, :] = point
        y_data[point_i, :] = t.evaluate(point)
      buff.insert(x_data, y_data)

  def intify_task(self, task):
    if task == "square":
      return [vocab_to_int[task]]
    elif task[:3] == "add":
      val = str(int(round(float(task[4:]))))
      return [vocab_to_int["add"]] + [vocab_to_int[x] for x in val]
    elif task[:4] == "mult":
      val = str(int(round(float(task[5:]))))
      return [vocab_to_int["mult"]] + [vocab_to_int[x] for x in val]
    elif task[:7] == "permute":
      val = task[8:]
      return [vocab_to_int["permute"]] + [vocab_to_int[x] for x in val]
    elif task[:3] == "is_":
      if task[3] == "X":
        return [vocab_to_int[x] for x in task.split('_')] 
      else:
        return [vocab_to_int["is"], vocab_to_int[task[3:]]]
    else:
      raise ValueError("Unrecognized meta task: %s" % task)


offset = run_config["run_offset"]
n = run_config["num_runs"]
for run_i in range(offset, offset + n):
  np.random.seed(run_i)
  tf.random.set_seed(run_i)
  run_config["this_run"] = run_i

  model = poly_HoMM_model(run_config=run_config)
  model.run_training()

  tf.reset_default_graph()

Starting run 2 of /data2/lampinen/polynomials_results/


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
keep_dims is deprecated, use keepdims instead
0, 0.002384, 0.740195, 16.430767, 0.424354, 2.603928, 0.148042, 26.502865, 2.282110, 0.893393, 3.326874, 15.668917, 1.598328, 0.182250, 21.950197, 2.382319, 4.480787, 2.377062, 10.669199, 3.891226, 5.188591, 0.357832, 1.664736, 3.440365, 3.942622, 6.213111, 4.045290, 5.982134, 4.465858, 21.211182, 1.630417, 10.109174, 2.196126, 1.321166, 2.543421, 1.117290, 11.223800, 4.158557, 10.397460, 12.099381, 39.626087, 12.839933, 12.668077, 4.894108, 3.322295, 22.830364, 11.651661, 7.742769, 0.146165, 1.233983, 5.726868, 6.013631, 4.880062, 2.732149, 5.992162, 4.195365, 2.474986, 4.097885, 0.099337, 15.159775, 10.187566, 0.000105, 674.296570, 0.022779, 2.784542, 5.539401, 17.420597, 2.693572, 12.713852, 480.835419, 2.884979, 109.386856, 14.920521, 6.291725, 1.214071, 106.109261, 143.787659, 514.025391, 4.153897, 38.773705, 0.010127, 246.605225, 9.060003, 9.478893, 47.852577, 8.252814, 15.394958, 6.631051, 10.115807, 9.7277